# Cats 02 - Monoids and Semigroups

Today we are going to talk about _monoids_ and _semigroups_.  It is important to know that these terms are actually mathematical terms as this can help us to better understand their definition, and help us to correctly create our own.  

The definition that we have for a _monoid_ from [Wolfgram Alpha](http://mathworld.wolfram.com/Monoid.html) is: 
> a set that is closed under an __associative__ _binary operation_ and has an __identity__ element $I$ in $S$ such that for all $a$ in $S$, $Ia=aI=a$.  It can also be thought of as a __semigroup__ with an _identity_ element.

So to cover terms:

- __*associative*__: means that the operation can be run portions in any order and produce the same result  
  - $(1 + 2) + 3 = 1 + (2 + 3)$
  - Subtration is an example a non associative opration: $(1 - 2) - 3 \neq 1 - (2 - 3)$
  

- __*binary operation*__: a function (operation) that uses exactly two elements.  
  - $f(x,y)$


- __*identity*__: a element that when combined with another value $x$ returns the same value when run through a given _binary operation_
  
     $ \left( \begin{array}{cc}
         a & b \\
         d & c
       \end{array} \right)
     %
       \left( \begin{array}{cc}
         1 & 0 \\
         0 & 1
       \end{array} \right)
     %
       =
     %
       \left( \begin{array}{cc}
         a & b \\
         d & c
       \end{array} \right)
     $
     
### Math to code

So, understanding the math terms and math syntax is great an all, but how does this end up working in scala?  Lucky for us we have already talked about _type classes_ which is a good thing as...

> moniods and semigroups are our first type classes.  
> They allow a developer to add or combine values.  <sub>_description from the Scala with Cats book_</sub>

## Monoid programatic definition

Alright, now that we know a little more about what a monoid is, lets see if we can create one.  In this case we are going to create a monoid trait
that can be implemented to provide the implimentation for the identity (called `empty`) and for the associative binary operation (called `combine`).  



In [1]:
trait SimpleMonoid[A] {
    def combine(left: A, right: A): A
    def empty: A
}

defined trait SimpleMonoid

Looking over our definition, we can see a couple of key features. 

1. It uses a generic type $A$
2. The function definition for `combine` is $(A, A) => A$
3. The `emtpy` just returns an $A$

So now that we have the signature, we can create a concrete implementation.  However we will want to make sure that
we follow the laws of a monoid that were described above.

1. the `combine` function must be associative, so in code we will want to test that...

        SimpleMonoid[A].combine(x, SimpleMonoid[A].combine(y, z)) === SimpleMonoid[A].combine(SimpleMonoid[A].combine(x, y),z)
        

2. the `emtpy` function result must return the original value...

        SimpleMonoid[A].combine(x, empty) === SimpleMonoid[A].combine(empty, x) === x

Let's go ahead and create a simple implementation for addition of int types.

In [2]:
val addMonoid = new SimpleMonoid[Int] {
    def combine(left: Int, right: Int) = left + right
    def empty = 0
}

addMonoid.combine(1, addMonoid.combine(2, 3)) == addMonoid.combine(addMonoid.combine(1,2), 3)
addMonoid.combine(2, addMonoid.empty) == addMonoid.combine(addMonoid.empty, 2)
addMonoid.combine(2, addMonoid.empty) == 2

addMonoid: AnyRef with SimpleMonoid[Int] = ammonite.$sess.cmd1$Helper$$anon$1@38e7af05
res1_1: Boolean = true
res1_2: Boolean = true
res1_3: Boolean = true

Great, we have create an `addMonoid` that will work with ints and follows all the requirements of a monoid.  

Now, take some time and see if you can implement the code for exercise 2.3 _The Truth About Monoids_.  A solution is provided in the cell below and you are welcome to expand the cell to compare your solution with another one.  

### Exercise 2.3 The Truth About Monoids

The `Boolean` type has a number of possible operations that can take place, specifically and `and` operation and an `or` operation.  Below, create two different monoids that support each of those operations and validate that your solution is correct.  

In [3]:
// Example 2.3 - Your Solution
val andMonoid = new SimpleMonoid[Boolean] {
    def combine(left: Boolean, right: Boolean) = left && right
    def empty = true
} 

andMonoid.combine(true, andMonoid.combine(true, false)) == andMonoid.combine(andMonoid.combine(true, true), false)
andMonoid.combine(true, andMonoid.combine(true, false))

val orMonoid = new SimpleMonoid[Boolean] {
    def combine(left: Boolean, right: Boolean) = left || right
    def empty = false
}

andMonoid: AnyRef with SimpleMonoid[Boolean] = ammonite.$sess.cmd2$Helper$$anon$1@7a25707e
res2_1: Boolean = true
res2_2: Boolean = false
orMonoid: AnyRef with SimpleMonoid[Boolean] = ammonite.$sess.cmd2$Helper$$anon$2@2081ee31

In [ ]:
// Exercise 2.3 - Example Solution
val andMonoid = new SimpleMonoid[Boolean] {
    def combine(left: Boolean, right: Boolean) = left && right
    def empty = true
}

andMonoid.combine(true, andMonoid.combine(false, true)) == andMonoid.combine(andMonoid.combine(true, false), true)
andMonoid.combine(true, andMonoid.empty) == andMonoid.combine(andMonoid.empty, true)
andMonoid.combine(true, andMonoid.empty) == true

val orMonoid = new SimpleMonoid[Boolean] {
    def combine(left: Boolean, right: Boolean) = left || right
    def empty = false
}

orMonoid.combine(true, orMonoid.combine(false, true)) == orMonoid.combine(orMonoid.combine(true, false), true)
orMonoid.combine(true, orMonoid.empty) == orMonoid.combine(orMonoid.empty, true)
orMonoid.combine(false, orMonoid.empty) == false

## Semigroup definition

Talking about monoids shows us that there is not a lot required for a monoid to exist, however it does require an _identity_ element, and this may not exist for all types.  In this
case you can create a **Semigroup** which is defined as:
> a _monoid_ without the _identity element_.  

So, we can implement this in code by creating another trait that just requires the `combine` function to exists.  But, we also want to stay as **DRY** as possible, this means that
we can also adjust our `SimpleMonoid` to be defined as inheriting from our _semigroup_.  

Lets demonstrate this behavoir below using two traits.  

In [4]:
trait SemiGroup[A] {
    def combine(left: A, right: A): A
}

trait SimpleMonoid[A] extends SemiGroup[A] {
    def empty: A
}

defined trait SemiGroup
defined trait SimpleMonoid

It is important to note that certain operations require that an identity exist, this is usually to act as a terminator or starting point.  

A common example is the `cons` operator ( `::` ).  When working with the cons operator you can add elements to a new list created using the _identity_ element which is represented by `Nil`.  

        3 :: 4 :: 5 :: Nil

### Exercise 2.4 All Set for Monoids

This next exercise is more of a thinking exercise.  If we have a `Set` there are certain operations that apply to a set, including `union`, `intersection`, etc. 

Think about these different operations and answer if it is a `monoid` or a `semigroup` or neither.  

## Monoids and Cats

One of the benefits of the cats library is that it provides many type classes for us already, this includes both the `Monoid` and the `Semigroup` type classes.  In this next section we are going to demonstrate how to use the supplied implementations for certains types (such as string) as well as creating an implementation for our own type.  

To start we will need to import the cats library calls.  

In [5]:
import $ivy.`org.typelevel::cats-core:2.0.0`

import cats.{Monoid, Semigroup}
import cats.instances.string._    // for Monoid[String]
import cats.instances.int._       // for Monoid[Int]
import cats.instances.option._    // for Monoid[Option[A]]
import cats.syntax.option._       // for x.some

import $ivy.$                               


import cats.{Monoid, Semigroup}

import cats.instances.string._    // for Monoid[String]

import cats.instances.int._       // for Monoid[Int]

import cats.instances.option._    // for Monoid[Option[A]]

import cats.syntax.option._       // for x.some

In [6]:
Monoid[Int].combine(1, 2)
Monoid[String].combine("Hi", "there")

Monoid[Int].combine(1, Monoid[Int].empty)

Monoid[Option[Int]].combine(1.some, 3.some)

res5_0: Int = 3
res5_1: String = "Hithere"
res5_2: Int = 1
res5_3: Option[Int] = Some(4)

### Nice operators

Since we are working with a more functional paradigm, it would be nice if there was an operator that we could use to make the code
more consise.  Luckily cats does provide some _syntatical sugar_ in the form of the `|+|` operator.

In [7]:
// Have to import in a different cell
import cats.syntax.semigroup._

import cats.syntax.semigroup._

In [8]:
"Hi " |+| "mis " |+| "amigos"
"Hi " |+| "mis " |+| "amigos" |+| Monoid[String].empty

res7_0: String = "Hi mis amigos"
res7_1: String = "Hi mis amigos"

### Our own Cat Monoid

Now that we have worked with the supplied monoid and semigroup instances from the cats library, lets create our own type and create a couple of _monoids_ or _semigroups_ that we want to use.  

To start we are going to create a `Cat` class. This class is pretty simple and only has two attributes:

- `name`: __String__
- `age`: __Int__

Lets start by defining the class and creating an `addCats` monoid.  

In [10]:
case class Cat(name: String, age: Int)

implicit val addCats: Monoid[Cat] = new Monoid[Cat] {
    val identityCat: Cat = Cat("", 0)
    
    // Build primitive to also use the monoid combine operation if possible
    def combine(left: Cat, right: Cat): Cat = Cat(left.name |+| right.name, left.age |+| right.age)
    def empty: Cat = identityCat
}

(Cat("me", 2) |+| (Cat("ow", 4) |+| Cat("er", 2))) == ((Cat("me", 2) |+| Cat("ow", 4)) |+| Cat("er", 2))
(addCats.empty |+| Cat("spot", 1)) == (Cat("spot", 1) |+| addCats.empty)
Cat("me", 2) |+| Cat("ow", 4)

defined class Cat
addCats: Monoid[Cat] = ammonite.$sess.cmd9$Helper$$anon$1@4c9cfbe2
res9_2: Boolean = true
res9_3: Boolean = true
res9_4: Cat = Cat("meow", 6)

## Where is this useful

There are a number of useful examples where having access to _semigroups_ and _monoids_ becomes useful.  

1. Big Data (Map/Reduce)
2. Distributed Operations
    - CRDTs